In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [9]:
import logging
from functools import partial
from dask.local import get_async
from dask import bag as db
from dask import array as da
import sprite2
import numpy as np
from matplotlib import pyplot as plt
import sprite2.aws
import sprite2.dask
import os

logging.debug("test")
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

### call a function remotely

In [10]:
def get_lambda_name():
    return os.environ.get('AWS_LAMBDA_FUNCTION_NAME', 'dummy')

In [11]:
# invoke locally, local environment doesnt define this var
get_lambda_name()

'dummy'

In [20]:
# this func will be executed remotely
get_remote_lambda_name = sprite2.aws.remote_invoke(get_lambda_name, request_id=10101)

In [21]:
get_remote_lambda_name()

'sprite'

### call a lambda remotely

In [22]:
# invoke remotely, lamba execution enfironment does
sprite2.aws.remote_invoke(lambda: 'im executing remotely')()

'im executing remotely'

### remote errors are raised locally

In [27]:
def fun(): 
    raise Exception('awha?')
    
sprite2.aws.remote_invoke(fun)()

AwsError: invalid response: {'errorMessage': 'awha?', 'errorType': 'Exception', 'stackTrace': [['/var/task/sprite2/aws.py', 87, 'executor', 'result = function()'], ['/Users/sayre/dev/sandbox/sprite2/sprite2/aws.py', 75, '<lambda>', ''], ['<ipython-input-27-c0fe0a063c87>', 2, 'fun', '']]}

### I can even numpy remotely

In [24]:
remote_square = sprite2.aws.remote_invoke(np.square)

In [25]:
%time remote_square(np.arange(10))

CPU times: user 10.7 ms, sys: 838 µs, total: 11.5 ms
Wall time: 196 ms


array([ 0,  1,  4,  9, 16, 25, 36, 49, 64, 81])

In [26]:
%timeit -n 5 remote_square(np.arange(10))

44.2 ms ± 10.8 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)
